In [ ]:
import pandas as pd
import pickle
import numpy as np
import os
from plotly import express as px
from plotly import graph_objects as go

In [ ]:
test_df = pd.read_csv('/mnt/famli_netapp_shared/C1_ML_Analysis/famli_ml_lists/dataset_list/2025-12-15/c3_instance_GA_test_dataset.csv')

df_rid9 = pd.read_csv('/mnt/raid/C1_ML_Analysis/FAMLI_app_test_dxa/runid_9_results.csv')

names = ['earlyGa', 'earlyGaScore', 'ga', 'gaScore']

df_farm = pd.DataFrame()

for name in names:
    df_temp = df_rid9[df_rid9['name'] == name][['studyid', 'value']].rename(columns={'value': name})
    if len(df_farm) == 0:
        df_farm = df_temp
    else:
        df_farm = df_farm.merge(df_temp, on='studyid', how='left')

df_farm['pid_date'] = df_farm['studyid'].str.split('-').str[0:3].str.join('-')      
df_farm['error_ga'] = df_farm['earlyGa'] - df_farm['ga']
df_farm['abs_ga'] = df_farm['error_ga'].abs()

In [ ]:
test_df.columns

In [ ]:
df_farm_merged = df_farm.merge(test_df[['session_id', 'ega']].drop_duplicates().rename(columns={'session_id': 'studyid'}), on='studyid', how='left')

In [ ]:
df_farm_merged.columns

In [ ]:
df_farm_merged['error'] = df_farm_merged['earlyGa'] - df_farm_merged['ega']
df_farm_merged['abs'] = df_farm_merged['error'].abs()

In [ ]:

px.violin(df_farm_merged.query('earlyGaScore > 0.5'), y='error', box=True, points="all")

In [ ]:
df_farm_merged['ega'].describe()

In [ ]:
with pd.option_context("display.max_rows", 200):
    display(df_farm_merged.query('abs > 20 and earlyGaScore > 0.5 and ega > 200'))

In [ ]:
print((df_farm_merged.error.abs() < 10).sum())

In [ ]:
px.scatter(df_farm_merged, x='earlyGa', y='ega', trendline='ols', title='Early GA vs GA from FAMLI App RunID 6')